<a href="https://colab.research.google.com/github/KSR-16/Life-Expectancy-Model/blob/main/Group_6_Life_Expectancy_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary packages
import pandas as pd
import statsmodels.api as sm
import statsmodels.tools
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle
pd.set_option('display.max_columns', None)

In [ ]:
# Read in train/test split data, set index equal to 1st column
X_train_fe = pd.read_csv('X_train_fe.csv', index_col=[0])
X_test_fe = pd.read_csv('X_test_fe.csv', index_col=[0])
y_train = pd.read_csv('y_train.csv', index_col=[0])
y_test = pd.read_csv('y_test.csv', index_col=[0])

# Squeeze y_train and y_test into panda.Series
y_train = y_train.squeeze()
y_test = y_test.squeeze()

# Check first 5 rows
X_train_fe.head()

,const,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Region_Africa,Region_Asia,Region_Central America and Caribbean,Region_European Union,Region_Middle East,Region_North America,Region_Oceania,Region_Rest of Europe,Region_South America
2026,1.0,-0.298246,-0.244969,-0.369456,0.137101,0.555556,0.551724,0.252033,0.3750,0.3750,-0.375,-0.045581,0.564994,2.051724,2.017241,0.759690,0,1,0,1,0,0,0,0,0,0,0
651,1.0,-0.403509,-0.327209,-0.348360,0.750979,0.500000,0.517241,0.552846,0.1875,0.3125,-0.200,0.855270,0.093200,-0.206897,-0.189655,0.813953,1,0,0,0,0,1,0,0,0,0,0
2225,1.0,-0.110276,-0.092738,-0.147051,0.369334,-0.611111,0.000000,0.552846,-0.8750,-0.9375,0.600,1.027696,0.977926,-0.293103,-0.310345,0.689922,0,1,0,0,0,0,0,0,0,0,1
2357,1.0,-0.200501,-0.174978,-0.581719,0.273083,0.555556,0.517241,0.512195,0.3750,0.3750,-0.325,-0.055739,-0.231884,-0.327586,-0.310345,0.635659,0,1,0,0,0,0,0,0,0,1,0
670,1.0,0.588972,0.894138,2.319636,0.128148,-0.333333,-0.655172,0.276423,-0.6875,-0.8750,23.950,-0.079962,-0.277815,1.879310,1.896552,0.364341,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
# Check first 5 rows
X_test_fe.head()

,const,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing,Region_Africa,Region_Asia,Region_Central America and Caribbean,Region_European Union,Region_Middle East,Region_North America,Region_Oceania,Region_Rest of Europe,Region_South America
1590,1.0,-0.131868,-0.111969,0.102023,0.603565,0.4375,0.0,0.603306,0.1250,0.1875,1.37500,3.081819,-0.374263,-0.173077,-0.169811,0.751938,0,1,0,0,1,0,0,0,0,0,0
1752,1.0,-0.417582,-0.347490,-0.726046,0.580824,-0.0625,0.1,0.595041,0.1875,0.1875,-0.21875,3.930425,3.071850,-0.500000,-0.547170,0.891473,1,0,0,0,0,0,0,0,0,1,0
772,1.0,0.793956,0.936293,0.415246,0.364474,-1.1875,-0.4,0.314050,0.0000,0.0000,2.90625,-0.297444,-0.383914,0.807692,0.792453,0.240310,0,1,0,0,1,0,0,0,0,0,0
1735,1.0,1.810440,2.206564,0.940366,0.088506,-1.0000,0.0,0.181818,-0.8750,-0.8750,2.75000,-0.315033,-0.023592,1.211538,1.169811,0.108527,0,1,1,0,0,0,0,0,0,0,0
387,1.0,-0.406593,-0.339768,-0.643476,0.559312,0.1875,-0.2,0.578512,0.1875,0.1875,-0.34375,1.746186,0.195710,-0.500000,-0.490566,0.651163,1,0,0,0,0,1,0,0,0,0,0


In [ ]:
# Check first 5 rows
y_train.head()

2026    76.1
651     75.7
2225    72.8
2357    76.6
670     50.6
Name: Life_expectancy, dtype: float64

In [ ]:
# Check first 5 rows
y_test.head()

1590    72.0
1752    81.3
772     62.7
1735    55.4
387     79.0
Name: Life_expectancy, dtype: float64

In [ ]:
# Check to see that the indexes match
print(sum(X_train_fe.index == y_train.index))
print(X_train_fe.shape)

2291
(2291, 27)


# First Model

In [ ]:
# Create preliminary model
lin_reg = sm.OLS(y_train, X_train_fe) # Use y_train and X_train data
results = lin_reg.fit() # Fit linear regression
results.summary() # Get summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     5836.
Date:                Thu, 18 Jan 2024   Prob (F-statistic):               0.00
Time:                        14:17:04   Log-Likelihood:                -3659.9
No. Observations:                2291   AIC:                             7370.
Df Residuals:                    2266   BIC:                             7513.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   44.8465      0.109    411.939      0.000      44.633      45.060
Infant_deaths                           -2.2748      0.255     -8.909      0.000      -2.776      -1.774
Under_five_deaths                       -2.7271      0.230    -11.860      0.000      -3.178      -2.276
Adult_mortality                         -6.6979      0.089    -74.993      0.000      -6.873      -6.523
Alcohol_consumption                     -0.1736      0.206     -0.843      0.400      -0.578       0.230
Hepatitis_B                             -0.1019      0.046     -2.213      0.027      -0.192      -0.012
Measles                                  0.0804      0.051      1.583      0.114      -0.019       0.180
BMI                                     -1.3819      0.279     -4.957      0.000      -1.929      -0.835
Polio                                    0.1138      0.094      1.210      0.226      -0.071       0.298
Diphtheria                              -0.1150      0.095     -1.208      0.227      -0.302       0.072
Incidents_HIV                            0.0372      0.007      5.045      0.000       0.023       0.052
GDP_per_capita                           0.2143      0.028      7.702      0.000       0.160       0.269
Population_mln                          -0.0041      0.004     -0.913      0.361      -0.013       0.005
Thinness_ten_nineteen_years             -0.1909      0.098     -1.956      0.051      -0.382       0.001
Thinness_five_nine_years                 0.1435      0.097      1.475      0.140      -0.047       0.334
Schooling                                1.4983      0.238      6.296      0.000       1.032       1.965
Economy_status_Developed                23.6769      0.115    206.021      0.000      23.452      23.902
Economy_status_Developing               21.1696      0.082    259.177      0.000      21.009      21.330
Region_Africa                            4.6130      0.090     51.344      0.000       4.437       4.789
Region_Asia                              4.9015      0.096     50.814      0.000       4.712       5.091
Region_Central America and Caribbean     6.5346      0.089     73.768      0.000       6.361       6.708
Region_European Union                    3.9160      0.126     31.189      0.000       3.670       4.162
Region_Middle East                       4.7458      0.108     43.914      0.000       4.534       4.958
Region_North America                     5.2825      0.187     28.227      0.000       4.916       5.650
Region_Oceania                           3.7374      0.106     35.226      0.000       3.529       3.945
Region_Rest of Europe                    4.8158      0.094     51.388      0.000       4.632       5.000
Region_South America                  

In [ ]:
y_pred = results.predict(X_train_fe) # Get y_pred using predict()
rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred) # Calculate RMSE
print(f'RMSE: {rmse}')

RMSE: 1.195516563383316


- Very high condition number, strong multicollinearity problems
- Many features have higher than 0.05 p-values
- RMSE is good - 1.20

# Second Model
## Dropping columns by their VIF

In [ ]:
# Get all the column names
cols = X_train_fe.columns

# Create an indexed list (a series) where we list the VIF of each of the columns
pd.Series([variance_inflation_factor(X_train_fe[cols].values, i) for i in range(X_train_fe[cols].shape[1])], index = X_train_fe[cols].columns)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


const                                    0.000000
Infant_deaths                           49.942936
Under_five_deaths                       51.674364
Adult_mortality                          8.298276
Alcohol_consumption                      3.353434
Hepatitis_B                              2.743322
Measles                                  1.655160
BMI                                      3.948867
Polio                                   12.553064
Diphtheria                              13.738433
Incidents_HIV                            3.066573
GDP_per_capita                           2.750356
Population_mln                           1.198752
Thinness_ten_nineteen_years              8.669616
Thinness_five_nine_years                 8.948924
Schooling                                5.497600
Economy_status_Developed                      inf
Economy_status_Developing                     inf
Region_Africa                                 inf
Region_Asia                                   inf


- VIF is quite high for a few columns, shows multicollinearity
- E.g. Infant_deaths and Under_five_deaths both have VIFs around 50, makes sense as they are extremely correlated - Pearson coefficient 0.95

In [ ]:
# Define function that drops columns if VIF scores are higher than 5 in a stepwise manner
def calculate_vif(X, thresh = 5.0):
    variables = list(range(X.shape[1])) # List of number of rows
    dropped = True
    while dropped:
        dropped = False
        # List comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif)) # Get the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # Delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # If we deleted anything, we set the 'dropped' value to True to stay in the while loop

            print('Remaining variables:')
            print(X.columns[variables]) # Finally, print the variables that are still in our set
    return X.iloc[:, variables] # Return our X cut down to the remaining variables

In [ ]:
# Run VIF function on X_train
X_train_fe_2 = calculate_vif(X_train_fe[['Infant_deaths', 'Under_five_deaths', 'Adult_mortality',
                                         'Alcohol_consumption', 'Hepatitis_B', 'Measles', 'BMI', 'Polio',
                                         'Diphtheria', 'Incidents_HIV', 'GDP_per_capita', 'Population_mln',
                                         'Thinness_ten_nineteen_years', 'Thinness_five_nine_years', 'Schooling',
                                         'Economy_status_Developed', 'Economy_status_Developing',
                                         'Region_Africa', 'Region_Asia', 'Region_Central America and Caribbean',
                                         'Region_European Union', 'Region_Middle East', 'Region_North America',
                                         'Region_Oceania', 'Region_Rest of Europe', 'Region_South America']])

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'Economy_status_Developed' at index: 15
Remaining variables:
Index(['Infant_deaths', 'Under_five_deaths', 'Adult_mortality',
       'Alcohol_consumption', 'Hepatitis_B', 'Measles', 'BMI', 'Polio',
       'Diphtheria', 'Incidents_HIV', 'GDP_per_capita', 'Population_mln',
       'Thinness_ten_nineteen_years', 'Thinness_five_nine_years', 'Schooling',
       'Economy_status_Developing', 'Region_Africa', 'Region_Asia',
       'Region_Central America and Caribbean', 'Region_European Union',
       'Region_Middle East', 'Region_North America', 'Region_Oceania',
       'Region_Rest of Europe', 'Region_South America'],
      dtype='object')
dropping 'Under_five_deaths' at index: 1
Remaining variables:
Index(['Infant_deaths', 'Adult_mortality', 'Alcohol_consumption',
       'Hepatitis_B', 'Measles', 'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV',
       'GDP_per_capita', 'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Devel

In [ ]:
# Add back the constant
X_train_fe_2 = sm.add_constant(X_train_fe_2)

## Dropping columns by their p-value

In [ ]:
# Define function that drops columns by their p-value in a stepwise manner
def stepwise_selection(X, y, threshold_in = 0.01, threshold_out = 0.05, verbose = True):
    # The function is checking for p-values (whether features are statistically significant) - lower is better

    included = [] # This is going to be the list of features we keep

    while True:
        changed = False
        ''' Forward step '''
        excluded = list(set(X.columns) - set(included)) # Get list of excluded columns
        new_pval = pd.Series(index = excluded, dtype = 'float64') # Create empty series
        for new_column in excluded: # Iterate through each excluded column
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit() # Fit model using included columns and new_column
            new_pval[new_column] = model.pvalues[new_column] # Put p-value of each column into series
        best_pval = new_pval.min() # Get the best p-value
        # Add the feature with the lowest (best) p-value under the threshold to our 'included' list
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin() # 'Lowest' p-value
            included.append(best_feature) # Append feature to 'included' list
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval)) # Specifying the verbose text


        ''' Backward step: removing features if new features added to the list make them statistically insignificant '''
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit() # Fit model using all included columns
        # Use all coefs except intercept
        pvalues = model.pvalues.iloc[1:] # Get all p-values
        worst_pval = pvalues.max() # Null if pvalues is empty
        # If the p-value exceeds the upper threshold, the feature will be dropped from the 'included' list
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval)) # Specifying the verbose text
        if not changed:
            break
    return included

In [ ]:
# Run stepwise_selection() on remaining features
final_columns = stepwise_selection(X_train_fe_2[list(X_train_fe_2.columns)], y_train)
print('resulting features:')
print(final_columns)

Add  const                          with p-value 0.0
Add  Infant_deaths                  with p-value 0.0
Add  Incidents_HIV                  with p-value 1.777e-254
Add  GDP_per_capita                 with p-value 4.77357e-110
Add  Region_Rest of Europe          with p-value 1.19914e-11
Add  Region_Asia                    with p-value 7.90394e-08
Add  Region_Central America and Caribbean with p-value 2.31337e-10
Add  Region_South America           with p-value 6.89539e-13
Add  Region_European Union          with p-value 0.00011279
Add  Region_Middle East             with p-value 1.71506e-06
Drop Region_Rest of Europe          with p-value 0.748857
Add  Region_North America           with p-value 2.15496e-09
Add  Measles                        with p-value 0.00475976
Add  Hepatitis_B                    with p-value 0.00647118
resulting features:
['const', 'Infant_deaths', 'Incidents_HIV', 'GDP_per_capita', 'Region_Asia', 'Region_Central America and Caribbean', 'Region_South America', '

In [ ]:
# Set dataset to remaining columns after VIF and p-value stepwise dropping
X_train_fe_final = X_train_fe_2[final_columns]

In [ ]:
# Create model
lin_reg = sm.OLS(y_train, X_train_fe_final) # Use y_train and X_train data
results = lin_reg.fit() # Fit linear regression
results.summary() # Get summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     2951.
Date:                Thu, 18 Jan 2024   Prob (F-statistic):               0.00
Time:                        14:39:24   Log-Likelihood:                -5282.0
No. Observations:                2291   AIC:                         1.059e+04
Df Residuals:                    2279   BIC:                         1.066e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
const                                   70.4731      0.105    669.633      0.000      70.267      70.679
Infant_deaths                           -9.6790      0.119    -81.567      0.000      -9.912      -9.446
Incidents_HIV                           -0.3851      0.009    -40.750      0.000      -0.404      -0.367
GDP_per_capita                           1.0202      0.043     23.527      0.000       0.935       1.105
Region_Asia                              1.8507      0.160     11.554      0.000       1.537       2.165
Region_Central America and Caribbean     2.1356      0.189     11.275      0.000       1.764       2.507
Region_South America                     2.3190      0.218     10.651      0.000       1.892       2.746
Region_European Union                    1.3678      0.185      7.405      0.000       1.006       1.730
Region_Middle East                       1.2920      0.212      6.085      0.000       0.876       1.708
Region_North America                     2.2060      0.407      5.414      0.000       1.407       3.005
Measles                                  0.3293      0.098      3.375      0.001       0.138       0.521
Hepatitis_B                             -0.1919      0.070     -2.725      0.006      -0.330      -0.054
==============================================================================
Omnibus:                       28.235   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.723
Skew:                          -0.040   Prob(JB):                     4.34e-11
Kurtosis:                       3.702   Cond. No.                         51.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
y_pred = results.predict(X_train_fe_final) # Get y_pred using predict()
rmse = statsmodels.tools.eval_measures.rmse(y_train, y_pred) # Calculate RMSE
print(f'RMSE: {rmse}')

RMSE: 2.4269232061328565


- All p-values are < 0.05
- Condition number - 51.6 (No multicollinearity)
- RMSE - 2.43

# Test Model on Test Data

In [ ]:
y_test_pred = results.predict(X_test_fe[final_columns]) # Use only the features left after dropping
rmse = statsmodels.tools.eval_measures.rmse(y_test, y_test_pred)
print(f'RMSE: {rmse}')

RMSE: 2.7244727827751523


In [ ]:
#with open('model.pkl', 'wb') as f:
    #pickle.dump(results, f)